# Projeto 1 - Ciência dos Dados

Nome: Vinícius de Barros Gomes

Nome: Gabriel Nakano Fujihara

Nome: Pedro Roberto Fernandes Noronha

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [90]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re
from unidecode import unidecode

In [91]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\gabri\OneDrive - Insper - Institudo de Ensino e Pesquisa\Insper\2023\1-semestre\DSoft(desing de software)\DataScienceProject


Carregando a base de dados com os tweets classificados manualmente:

In [92]:
train = pd.read_excel('dados_treino.xlsx')
train.head(5)
train['Mensagem']=train['Mensagem'].astype('category')

In [93]:
test = pd.read_excel('dados_teste.xlsx')
test.head(5)

,Mensagem,"Target (Neutro, Positivo, Negativo)"
0,"Apesar da narrativa prender o leitor, os argum...",negativo
1,A formatação desse ebook atrapalha muito a lei...,neutro
2,Não é o primeiro livro da Record que oxida em ...,neutro
3,O hype que envolve esse livro é ENORME então e...,negativo
4,"meu deus, essa escritora conseguiu estragar um...",negativo


___
## Classificador automático


Faça aqui uma descrição do seu assunto e o que considerou como relevante ou não relevante na classificação dos tweets (Target).

Buscamos avaliar se os livros oferecidos pela amazon são ou não bons, portanto, foi ignorado quando haviam críticas exclusivamente ao serviço da amazon, como entrega e ou qualidade física do produto.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [94]:
def cleanup(text):
    text=text.lower()
    punctuation = '[´"!-.:?;$''<>]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

def remover_acentos(texto):
    # Use unidecode para remover acentos e converter caracteres especiais para caracteres ASCII
    texto_sem_acentos = unidecode(texto)
    return texto_sem_acentos


In [95]:
def df0(df):
    df['Mensagem'] = df['Mensagem'].apply(cleanup)
    df['Mensagem'] = df['Mensagem'].apply(remover_acentos)
    novo_df = pd.concat([df[['Mensagem']], df[['Target (Loja e Sistema, Entrega,Conteudo)']]], axis=1)
    return novo_df
train=df0(train)
train['Mensagem']=train['Mensagem'].astype('category')

In [96]:
df_Loja_Sistema=train.loc[train["Target (Loja e Sistema, Entrega,Conteudo)"]=='Loja e Sistema']
df_Entrega=train.loc[train["Target (Loja e Sistema, Entrega,Conteudo)"]=='Entrega']
df_Conteudo=train.loc[train["Target (Loja e Sistema, Entrega,Conteudo)"]=='Conteúdo do Livro']
quant_loja=len(df_Loja_Sistema)
quant_Conteudo=len(df_Conteudo)
quant_Entrega=len(df_Entrega)


In [97]:
def ll(lista):
    todas_as_frases=lista["Mensagem"]
    todas_as_palavras = []
    for i in todas_as_frases:
        frase=[]
        x=i.split(" ")
        for u in x:
            if u not in frase:
                frase.append(u)
        for d in frase:
            todas_as_palavras.append(d)
    todas_as_palavrasSeries=pd.Series(todas_as_palavras)
    return todas_as_palavrasSeries.value_counts()
quant_palavras_Entrega= ll(df_Entrega)
quant_palavras_Conteudo= ll(df_Conteudo)
quant_palavras_Loja= ll(df_Loja_Sistema)

In [107]:
quantidade_total=quant_loja + quant_Conteudo + quant_Entrega
def laplace(x,q):
    return (x+1)/(q+quantidade_total)

In [110]:
frase=str(input('frase:'))
frase=cleanup(frase)
frase=remover_acentos(frase)
frase=frase.lower()
palavras=frase.split(' ')
prob_Loja_Sistema=1
prob_Entrega=1
prob_Conteudo=1
for palavra in palavras:
    if palavra in quant_palavras_Loja:
        prob_Loja_Sistema*=(quant_palavras_Loja[palavra]/quant_loja)
    else:
        prob_Loja_Sistema*=laplace(0,quant_loja)
for palavra in palavras:
    if palavra in quant_palavras_Entrega:
        prob_Entrega*=(quant_palavras_Entrega[palavra]/quant_Entrega)
    else:
        prob_Entrega*=laplace(0,quant_Entrega)
for palavra in palavras:
    if palavra in quant_palavras_Conteudo:
        prob_Conteudo*=(quant_palavras_Conteudo[palavra]/quant_Conteudo)
    else:
        prob_Conteudo*=laplace(0,quant_Conteudo)
print(prob_Conteudo)
print(prob_Entrega)
print(prob_Loja_Sistema)
maior_prob=max(prob_Conteudo , prob_Entrega , prob_Loja_Sistema)
if maior_prob==prob_Conteudo:
    print("Conteúdo do Livro")
elif maior_prob==prob_Entrega:
    print("Entrega")
elif maior_prob==prob_Loja_Sistema:
    print("Loja e Sistema")

0.0008453085376162299
0.001184834123222749
0.0011668611435239206
Entrega


In [116]:
def classifica(df):
    df_Loja_Sistema=train.loc[train["Target (Loja e Sistema, Entrega,Conteudo)"]=='Loja e Sistema']
    df_Entrega=train.loc[train["Target (Loja e Sistema, Entrega,Conteudo)"]=='Entrega']
    df_Conteudo=train.loc[train["Target (Loja e Sistema, Entrega,Conteudo)"]=='Conteúdo do Livro']
    quant_loja=len(df_Loja_Sistema)
    quant_Conteudo=len(df_Conteudo)
    quant_Entrega=len(df_Entrega)
    quant_palavras_Entrega= ll(df_Entrega)
    quant_palavras_Conteudo= ll(df_Conteudo)
    quant_palavras_Loja= ll(df_Loja_Sistema)
    quantidade_total=quant_loja + quant_Conteudo + quant_Entrega
    df['Mensagem']=df['Mensagem'].astype('category')
    df=df0(df)
    for frase in df["Mensagem"]:
        frase=cleanup(frase)
        frase=remover_acentos(frase)
        frase=frase.lower()
        palavras=frase.split(' ')
        prob_Loja_Sistema=1
        prob_Entrega=1
        prob_Conteudo=1
        for palavra in palavras:
            if palavra in quant_palavras_Loja:
                prob_Loja_Sistema*=(quant_palavras_Loja[palavra]/quant_loja)
            else:
                prob_Loja_Sistema*=laplace(0,quant_loja)
        for palavra in palavras:
            if palavra in quant_palavras_Entrega:
                prob_Entrega*=(quant_palavras_Entrega[palavra]/quant_Entrega)
            else:
                prob_Entrega*=laplace(0,quant_Entrega)
        for palavra in palavras:
            if palavra in quant_palavras_Conteudo:
                prob_Conteudo*=(quant_palavras_Conteudo[palavra]/quant_Conteudo)
            else:
                prob_Conteudo*=laplace(0,quant_Conteudo)
        maior_prob=max(prob_Conteudo , prob_Entrega , prob_Loja_Sistema)
        if maior_prob==prob_Conteudo:
            return "Conteúdo do Livro"
        elif maior_prob==prob_Entrega:
            return "Entrega"
        elif maior_prob==prob_Loja_Sistema:
            return "Loja e Sistema"

print(classifica(test))

KeyError: "None of [Index(['Target (Loja e Sistema, Entrega,Conteudo)'], dtype='object')] are in the [columns]"

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CONSIDEROU mais de duas categorias na variável Target e INCREMENTOU a quantidade de notícias, mantendo pelo menos 250 notícias por categoria (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* Para Target com duas categorias: CRIOU pelo menos quatro categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto (pelo menos dois cenários, exceto aqueles já apresentados em sala pelos professores: por exemplo, filtro de spam)
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item Qualidade do Classificador a partir de novas separações das Notícias entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**